In [ ]:
from osgeo import gdal

In [ ]:
import os
from utils.GenUtils import make_folder, get_paths, chunk_creator, folder_file_size, question

In [ ]:
os.environ["ISISROOT"]="/opt/conda/envs/isis/"
os.environ["ISISDATA"]="/isis-data"

In [ ]:
import kalasiris as isis

In [ ]:
config = {
            "inst":"lronac",
            "maptemplate":"/home/jovyan/scripts/PyISIS-Parallel/maptemplates/Center_EquirectangularMoon.map",
            "oxt":"tiff",
            "PATH":"/home/jovyan/shoggoth/Data_challenge/Raw_and_proccesings/Boulder_LROC_NAC_Data/",
            "DSTPATH":"/home/jovyan/shoggoth/Data_challenge/Raw_and_proccesings/Boulder_LROC_NAC_Data/",
            "byte":"n"
            }

In [ ]:
if config['inst']=='lronac':
    from utils.KalaUtils import lro
    proc_func=lro
    cam = 'nac'
    ixt='IMG'
elif config['inst']=='lrowac':
    from utils.KalaUtils import lro
    proc_func=lro
    cam = 'wac'
    ixt='IMG'
elif config['inst']=='m3L1':
    from utils.KalaUtils import m3L1
    proc_func=m3L1
    cam = None
    ixt='LBL'
elif config['inst']=='m3L2':
    from utils.KalaUtils import m3L2
    proc_func=m3L2
    cam = None
    ixt='LBL'
else:
    print('Cannot find utils')
    
maptemplate=config['maptemplate']
byte = config['byte']

In [ ]:
def mapper(src, proc_func, maptemplate, ixt, oxt, cam, byte):
    src_basename = os.path.basename(src).split('.'+ixt)[0]
    dst_basename = DSTPATH+'/'+src_basename
    print(dst_basename)
#    if os.path.isfile(dst_basename+'_lev2.JP2'):
#        print ("File exist")
#    elif os.path.isfile(dst_basename+'.JP2'):
#        print ("File exist")
#    else:
    try:
        proc_func(src, dst_basename, maptemplate, config['oxt'], cam, byte)
    except Exception as e:
        print(e)
        
  

In [ ]:
def parallel_mapper(files, JOBS, proc_func, maptemplate, ixt, oxt, cam, byte):
    from joblib import Parallel, delayed, parallel_backend
    with parallel_backend("loky", inner_max_num_threads=2):

        Parallel (n_jobs=JOBS)(delayed(mapper)(files[i],proc_func, maptemplate, ixt, oxt, cam, byte)
                                for i in range(len(files)))

In [ ]:
def main(PATH, DPATH, ixt, maptemplate):
    image_list = get_paths(PATH, ixt) 
    total_size, max_size, av_fsize = folder_file_size(PATH,image_list)
    from tqdm import tqdm
    import psutil
    avram=psutil.virtual_memory().total >> 30
    avcores=psutil.cpu_count(logical=False)
    avthreads=psutil.cpu_count(logical=True)
    ram_thread = avram/avthreads
    req_mem = avthreads*max_size
    if req_mem > avcores and req_mem > avram:
        JOBS = int(avcores - avcores/(avram/avcores))
    else:
        JOBS = int(avcores - avcores/(avram/avcores))
    #JOBS = 1
    with tqdm(total=len(image_list),
             desc = 'Generating Images',
             unit='File') as pbar:
        filerange = len(image_list)
        chunksize = round(filerange/JOBS)
        if chunksize <1:
            chunksize=1
            JOBS = filerange
        chunks = []
        for c in chunk_creator(image_list, JOBS):
            chunks.append(c)
        # from time import time
        from datetime import datetime
        for i in range(len(chunks)):
            start = datetime.now()
            dt_string = start.strftime("%d/%m/%Y %H:%M:%S")
            #print(f'Loop {i} started at: {dt_string}', chunks[i])
            files = chunks[i]
            parallel_mapper(files, JOBS, proc_func, maptemplate, ixt, config['oxt'], cam, byte)
            pbar.update(JOBS)

In [ ]:
if __name__ == "__main__":
    PATH=config['PATH']
    DSTPATH=config['DSTPATH']
    os.makedirs(DSTPATH, exist_ok=True)
    print(ixt)
    main(PATH, DSTPATH, ixt, maptemplate)